In [1]:
from awpy import Demo

import pandas as pd
import numpy as np

import seaborn as sns

import random
import sys
import os

pd.set_option('display.max_columns', 100)
sys.path.append(os.path.abspath('./package'))

from CS2_tabular_graph_data_creator import CS2TabularGraphDataCreator

### ***awpy2*** test

In [2]:
player_cols = [
    'X',
    'Y',
    'Z',
    'health',
    'score',
    'mvps',
    'is_alive',
    'balance',
    'inventory',
    'life_state',
    'pitch',
    'yaw',
    'armor',
    'has_defuser',
    'has_helmet',
    'player_name',
    'start_balance',
    'total_cash_spent',
    'cash_spent_this_round',
    'move_collide',
    'move_type',
    'team_num',
    'active_weapon',
    'jump_velo',
    'fall_velo',
    'in_crouch',
    'crouch_state',
    'ducked',
    'ducking',
    'in_duck_jump',
    'spotted',
    'approximate_spotted_by',
    'time_last_injury',
    'player_state',
    'passive_items',
    'is_scoped',
    'is_walking',
    'resume_zoom',
    'is_defusing',
    'in_bomb_zone',
    'move_state',
    'which_bomb_zone',
    'in_hostage_rescue_zone',
    'stamina',
    'direction',
    'armor_value',
    'velo_modifier',
    'flash_duration',
    'flash_max_alpha',
    'round_start_equip_value',
    'current_equip_value',
    'velocity',
    'velocity_X',
    'velocity_Y',
    'velocity_Z',
]

other_cols = [
    'num_player_alive_ct',
    'num_player_alive_t',
    'ct_losing_streak',
    't_losing_streak',
    'active_weapon_name',
    'active_weapon_ammo',
    'total_ammo_left',
    'is_in_reload',
    'alive_time_total',
    'is_bomb_dropped'
]

dem = Demo(path='../../data/matches-raw/cs2/2024/_blast-premier-spring-final-2024-faze-vs-vitality-bo3-lQLvdLlJcL7dI4fNDLSZOXfaze-vs-vitality-m3-mirage.dem', \
           player_props=player_cols, other_props=other_cols)

In [12]:
dem.events.keys()

dict_keys(['round_freeze_end', 'round_prestart', 'bomb_exploded', 'player_ping', 'bomb_defused', 'player_given_c4', 'round_announce_match_start', 'begin_new_match', 'round_announce_warmup', 'player_connect_full', 'item_equip', 'hltv_fixed', 'round_poststart', 'entity_killed', 'hltv_versioninfo', 'smokegrenade_detonate', 'vote_cast', 'player_sound', 'player_ping_stop', 'inferno_startburn', 'cs_pre_restart', 'bomb_planted', 'item_pickup', 'cs_win_panel_match', 'flashbang_detonate', 'smokegrenade_expired', 'bomb_pickup', 'round_end_upload_stats', 'player_hurt', 'bomb_dropped', 'player_footstep', 'weapon_fire', 'round_announce_last_round_half', 'player_death', 'player_jump', 'cs_round_start_beep', 'hltv_chase', 'weapon_zoom', 'player_disconnect', 'round_time_warning', 'cs_round_final_beep', 'inferno_expire', 'round_officially_ended', 'weapon_reload', 'announce_phase_end', 'player_spawn', 'round_announce_match_point', 'cs_intermission', 'other_death', 'hegrenade_detonate', 'server_cvar', 'p

### Graph creator test

In [3]:
tc = CS2TabularGraphDataCreator()
tc.MATCH_PATH = '../../data/matches-raw/cs2/2024/_iem-dallas-2024-g2-vs-mouz-bo3-O1BgM30a6fWPjk5gU5VUnXg2-vs-mouz-m2-nuke.dem'
tc.PLAYER_STATS_DATA_PATH = ''
tc.MISSING_PLAYER_STATS_DATA_PATH = ''

ticks, kills, rounds, bomb, damages, smokes, infernos = tc._INIT_get_needed_dataframes()
pf = tc._PLAYER_calculate_ingame_features_from_needed_dataframes(ticks, kills, rounds, damages)
pf = tc._PLAYER_get_activeWeapon_dummies(pf)

In [4]:
pf

,tick,round,game_time,team_name,name,X,Y,Z,pitch,yaw,velocity_X,velocity_Y,velocity_Z,health,armor_value,active_weapon_name,active_weapon_ammo,total_ammo_left,is_alive,in_crouch,ducking,in_duck_jump,is_walking,spotted,is_scoped,is_defusing,is_in_reload,flash_duration,in_bomb_zone,balance,current_equip_value,round_start_equip_value,ct_losing_streak,t_losing_streak,is_bomb_dropped,freeze_end,end,winner,is_CT,stat_kills,stat_HS_kills,stat_opening_kills,stat_deaths,stat_opening_deaths,stat_assists,stat_flash_assists,stat_damage,stat_weapon_damage,stat_nade_damage,stat_survives,stat_KPR,stat_ADR,stat_DPR,stat_HS%,stat_SPR,activeWeapon_C4,activeWeapon_Knife,activeWeapon_Taser,activeWeapon_USP-S,activeWeapon_P2000,activeWeapon_Glock-18,activeWeapon_Dual Berettas,activeWeapon_P250,activeWeapon_Tec-9,activeWeapon_CZ75 Auto,activeWeapon_Five-SeveN,activeWeapon_Desert Eagle,activeWeapon_MAC-10,activeWeapon_MP9,activeWeapon_MP7,activeWeapon_MP5-SD,activeWeapon_UMP-45,activeWeapon_PP-Bizon,activeWeapon_P90,activeWeapon_Nova,activeWeapon_XM1014,activeWeapon_Sawed-Off,activeWeapon_MAG-7,activeWeapon_M249,activeWeapon_Negev,activeWeapon_FAMAS,activeWeapon_Galil AR,activeWeapon_AK-47,activeWeapon_M4A4,activeWeapon_M4A1,activeWeapon_SG 553,activeWeapon_AUG,activeWeapon_SSG 08,activeWeapon_AWP,activeWeapon_G3SG1,activeWeapon_SCAR-20,activeWeapon_HE Grenade,activeWeapon_Flashbang,activeWeapon_Smoke Grenade,activeWeapon_Incendiary Grenade,activeWeapon_Molotov,activeWeapon_Decoy Grenade
0,6338,1,982.578125,CT,NiKo,2585.000000,-344.000000,-351.96875,-0.973328,-169.127655,0.000000,0.000000,0.0,100,100,Knife,0.0,1.0,True,None,None,None,False,False,False,False,False,0.000000,False,150,850,200,1,1,False,6338,10621,CT,0,0,0,0,0,0,0,0,0,0,0,1,0.000000,0.000000,0.000000,NaN,1.000000,0,1,0.0,0,0,0,0,0,0,0.0,0,0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0.0
1,6338,1,982.578125,CT,m0NESY,2584.000000,-504.000000,-351.96875,3.137283,-179.670746,0.000000,0.000000,0.0,100,100,Knife,0.0,1.0,True,None,None,None,False,False,False,False,False,0.000000,False,150,850,200,1,1,False,6338,10621,CT,0,0,0,0,0,0,0,0,0,0,0,1,0.000000,0.000000,0.000000,NaN,1.000000,0,1,0.0,0,0,0,0,0,0,0.0,0,0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0.0
2,6338,1,982.578125,CT,Stewie2K,2504.000000,-344.000000,-351.96875,2.518280,-171.692276,0.000000,0.000000,0.0,100,100,Knife,0.0,1.0,True,None,None,None,False,False,False,False,False,0.000000,False,150,850,200,1,1,False,6338,10621,CT,0,0,0,0,0,0,0,0,0,0,0,1,0.000000,0.000000,0.000000,NaN,1.000000,0,1,0.0,0,0,0,0,0,0,0.0,0,0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0.0
3,6338,1,982.578125,TERRORIST,Brollan,-1808.000000,-1089.000000,-415.96875,-6.520721,-6.717453,0.000000,0.000000,0.0,100,0,Glock-18,20.0,0.0,True,None,None,None,False,False,False,False,False,0.000000,False,0,700,200,1,1,False,6338,10621,CT,0,0,0,0,0,0,0,0,0,0,0,1,0.000000,0.000000,0.000000,NaN,1.000000,0,0,0.0,0,0,1,0,0,0,0.0,0,0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0.0
4,6338,1,982.578125,TERRORIST,torzsi,-1947.010010,-965.109985,-415.96875,5.565948,-39.881744,0.000000,0.000000,0.0,100,100,Knife,0.0,1.0,True,None,None,None,False,False,False,False,False,0.000000,False,150,850,200,1,1,False,6338,10621,CT,0,0,0,0,0,0,0,0,0,0,0,1,0.000000,0.000000,0.000000,NaN,1.000000,0,1,0.0,0,0,0,0,0,0,0.0,0,0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053635,152800,1